In [1]:
import pop2net as p2n
import mesa
import agentpy as ap

# Integration into ABM frameworks

Pop2net provides seamless integration with two widely used agent-based modeling (ABM) frameworks: Mesa and AgentPy.

To use Pop2net with either framework, you only need to follow two steps:

First, when defining an Actor or Location class, it must inherit from both the corresponding Pop2net class and the framework’s class — in that order. For example:

```python
class Actor(p2n.Actor, mesa.Agent):
    pass

class Location(p2n.Location, mesa.Agent):
    pass
```

Second, provide the model instance and the name of the framework to the Environment object:

```python
env = p2n.Environment(model=model, framework="mesa")
```

## A Simple Example with Mesa

Let's begin by defining an `Actor` class.  
It's important that the class inherits first from `p2n.Actor`, and then from `mesa.Agent`:

In [2]:
class Actor(p2n.Actor, mesa.Agent):
    def say_hello(self):
        print("Hello I am an actor.")

Location classes must also inherit from `mesa.Agent`:

In [3]:
class Location(p2n.Location, mesa.Agent):
    def say_hello(self):
        print("Hello I am a location.")

A common use case when integrating Pop2net with Mesa (or AgentPy) is to extend the simulation model with an `Environment` object. This object stores all actors and locations and manages their relationships using Pop2net's structure.
When extending a Mesa model with a Pop2net environment, it's important to pass the model instance to the `Environment` and to set the `framework` argument to `"mesa"`:


In [4]:
class Model(mesa.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # add pop2net environment object as model attribute
        self.env = p2n.Environment(model=self, framework="mesa")

        # add one actor to the environment
        self.env.add_actor(Actor(model=self))

        # add one location to the environment
        self.env.add_location(Location(model=self))

    def step(self):
        # Because env.actors and env.locations are AgentSets now,
        # we can use the Mesa Syntax to let the agents do something:
        self.env.actors.do("say_hello")
        self.env.locations.do("say_hello")

Let's create the model and have a look at the agent attribute:

In [5]:
model = Model()
list(model.agents)

[<__main__.Actor at 0x73cefae16b10>, <__main__.Location at 0x73cf500ae390>]

As you can see, the agent attribute includes both the actor and the location because locations are also considered as agents in Pop2net.
This is something you have to keep in mind!

To get only actors or only locations, you can use `model.env.actors` or `model.env.locations`. Both are `mesa.AgentSet`s now:

In [6]:
model.env.actors

In [7]:
list(model.env.actors)

In [8]:
model.env.locations

In [9]:
list(model.env.locations)

Let's run the model for one step:

In [10]:
model.step()

Hello I am an actor.
Hello I am a location.


## A simple example with AgentPy

Here is the same example adapted for **AgentPy**

In [11]:
class Actor(p2n.Actor, ap.Agent):
    def say_hello(self):
        print("Hello I am an actor.")

In [12]:
class Location(p2n.Location, ap.Agent):
    def say_hello(self):
        print("Hello I am a location.")

In [13]:
class Model(ap.Model):
    def setup(self):
        # add pop2net environment object as model attribute
        self.env = p2n.Environment(model=self, framework="agentpy")

        # add one actor to the environment
        self.env.add_actor(Actor(model=self))

        # add one location to the environment
        self.env.add_location(Location(model=self))

    def step(self):
        # Because env.actors and env.locations are AgentSets now,
        # we can use the Mesa Syntax to let the agents do something:
        self.env.actors.say_hello()
        self.env.locations.say_hello()

In [14]:
model = Model()
model.run(steps=1)

Hello I am an actor.
Hello I am a location.
Completed: 1 steps
Run time: 0:00:00.000297
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}